# Copy Shared Drive Content to MyDrive

> **Note:** Please read the README.md file for additional information.

This notebook helps you copy content from a shared Google Drive folder to your personal MyDrive folder.

## Step 1: Mount Google Drive

First, we need to allow Colab to access your Google Drive. Run the cell below to mount your drive.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

## Step 2: Verify Drive Access

Let's verify that the drive is mounted correctly by listing the contents of MyDrive.

In [ ]:
!ls -l "/content/drive/MyDrive"

## Step 3: Copy Folder Content

This step will copy the source folder to a new destination folder. If the destination folder already exists, it will be removed first.

In [ ]:
import shutil
import os
from pathlib import Path

source = str(input("Enter the name folder you want to copy: "))
dest = str(input("Enter the name folder you want to paste: "))

source_dir = f"/content/drive/MyDrive/{source}"
dest_dir = f"/content/drive/MyDrive/{dest}"


if not os.path.exists(source_dir):
    print(f"Error: Source directory does not exist: {source_dir}")
    print("Please verify the source path and try again.")
else:
    print(f"Source directory found: {source_dir}")
    if os.path.exists(dest_dir):
        print(f"Removing existing destination directory: {dest_dir}")
        shutil.rmtree(dest_dir)
    
    print(f"\nCopying from '{source_dir}' to '{dest_dir}'...")
    print("This may take a while depending on the folder size...\n")
    
    try:
        shutil.copytree(source_dir, dest_dir)
        print("Copy completed successfully!")
        

        if os.path.exists(dest_dir):
            source_size = sum(f.stat().st_size for f in Path(source_dir).rglob('*') if f.is_file())
            dest_size = sum(f.stat().st_size for f in Path(dest_dir).rglob('*') if f.is_file())
            print(f"\n Verification:")
            print(f"   Source size: {source_size / (1024**3):.2f} GB")
            print(f"   Destination size: {dest_size / (1024**3):.2f} GB")
    except Exception as e:
        print(f"Error during copy: {e}")
        print(f"Error type: {type(e).__name__}")
